[TODO]: About Mnist

## Creating the data lodaers

In [1]:
from core.dataLoaders.fashion_mnist import Fashion_MNIST

#instantiate the data class
data_cls = Fashion_MNIST()

# create data loader
[train_data, test_data] = data_cls.create_loader()

data_cls.view_sample_stats()

/Users/srrthk/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/srrthk/anaconda3/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /Users/srrthk/anaconda3/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <13D54172-DB68-3BCA-9DFD-9BABA030E1E4> /Users/srrthk/anaconda3/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(



------------- Train Stats --------------

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64

------------- test Stats  --------------

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64

-------------    END      ---------------

